<a href="https://colab.research.google.com/github/22ananya/MUSI6201/blob/main/Final%20Project%20-%20Stereo%20Demixing/enhance_single_track.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import generally required packages - update as needed

In [2]:
# Import dependencies
import numpy as np
import matplotlib.pyplot as plt
import librosa
import scipy.signal as sp
import scipy.io.wavfile as wav

Import/Install the prerequisite code for implementing the Cadenza challenge - includes baselines, other important files

In [3]:
!pip install pyclarity==0.4.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.6/750.6 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 73.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.9/776.9 kB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 65.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 19.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.7/74.7 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 67.3 MB/s eta 0:00:00
  Created wheel for antlr4-python3-runtime: file

Import the dataset for the Cadenza challenge directly through the Google Drive link - Only needs to be done once! So now commented out


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Check current path

In [6]:
import os
print(os.getcwd())

/content


Change current path to Cadenza folder

In [7]:
os.chdir('/content/drive/MyDrive/Cadenza_Challenge/cad_icassp_2024')
print(os.getcwd())

/content/drive/MyDrive/Cadenza_Challenge/cad_icassp_2024


#Process single audio file through the entire model step by step

Import all dependencies - same as enhance.py file

In [8]:
from __future__ import annotations

import json
import logging
from pathlib import Path

# pylint: disable=import-error
import hydra
import numpy as np
import torch
from numpy import ndarray
from omegaconf import DictConfig
from torchaudio.pipelines import HDEMUCS_HIGH_MUSDB

from clarity.enhancer.compressor import Compressor
from clarity.enhancer.nalr import NALR
from clarity.utils.audiogram import Listener
from clarity.utils.file_io import read_signal
from clarity.utils.flac_encoder import FlacEncoder
from clarity.utils.signal_processing import (
    clip_signal,
    denormalize_signals,
    normalize_signal,
    resample,
    to_16bit,
)
from clarity.utils.source_separation_support import get_device, separate_sources
from recipes.cad_icassp_2024.baseline.evaluate import (
    apply_gains,
    apply_ha,
    make_scene_listener_list,
    remix_stems,
)

logger = logging.getLogger(__name__)

/usr/local/lib/python3.10/dist-packages/recipes/cad_icassp_2024/baseline/evaluate.py:190: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path="", config_name="config")


Import all the required functions defined in Enhance.py that do not need to be changed

In [9]:
from recipes.cad_icassp_2024.baseline.enhance import (
    save_flac_signal,
    decompose_signal,
    process_remix_for_listener
)

/usr/local/lib/python3.10/dist-packages/recipes/cad_icassp_2024/baseline/enhance.py:182: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path="", config_name="config")


Import the correct config file (hardcoded location)

In [10]:
from omegaconf import OmegaConf
config = OmegaConf.load('baseline/config.yaml')
print(config.separator.model)

demucs


Set output directory

In [11]:
enhanced_folder = Path("enhanced_signals_single")
enhanced_folder.mkdir(parents=True, exist_ok=True)

#Load source sep model  - pretrained

In [12]:
if config.separator.model == "demucs":
    separation_model = HDEMUCS_HIGH_MUSDB.get_model()
    model_sample_rate = HDEMUCS_HIGH_MUSDB.sample_rate
    sources_order = separation_model.sources
    normalise = True

100%|██████████| 319M/319M [00:06<00:00, 47.9MB/s]


Configure device

In [13]:
device, _ = get_device(config.separator.device)
separation_model.to(device)

HDemucs(
  (freq_encoder): ModuleList(
    (0): _HEncLayer(
      (conv): Conv2d(4, 48, kernel_size=(8, 1), stride=(4, 1), padding=(2, 0))
      (norm1): Identity()
      (rewrite): Conv2d(48, 96, kernel_size=(1, 1), stride=(1, 1))
      (norm2): Identity()
      (dconv): _DConv(
        (layers): ModuleList(
          (0): Sequential(
            (0): Conv1d(48, 12, kernel_size=(3,), stride=(1,), padding=(1,))
            (1): GroupNorm(1, 12, eps=1e-05, affine=True)
            (2): GELU(approximate='none')
            (3): Conv1d(12, 96, kernel_size=(1,), stride=(1,))
            (4): GroupNorm(1, 96, eps=1e-05, affine=True)
            (5): GLU(dim=1)
            (6): _LayerScale()
          )
          (1): Sequential(
            (0): Conv1d(48, 12, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
            (1): GroupNorm(1, 12, eps=1e-05, affine=True)
            (2): GELU(approximate='none')
            (3): Conv1d(12, 96, kernel_size=(1,), stride=(1,))
           

Load listener data

In [14]:
config.path.root = '/content/drive/MyDrive/Cadenza_Challenge/cad_icassp_2024'
print(config.path)


{'root': '/content/drive/MyDrive/Cadenza_Challenge/cad_icassp_2024', 'metadata_dir': '${path.root}/metadata', 'music_dir': '${path.root}/audio/at_mic_music', 'gains_file': '${path.metadata_dir}/gains.json', 'head_positions_file': '${path.metadata_dir}/head_positions.json', 'listeners_file': '${path.metadata_dir}/listeners.train.json', 'music_file': '${path.metadata_dir}/at_mic_music.train.json', 'scenes_file': '${path.metadata_dir}/scenes.train.json', 'scene_listeners_file': '${path.metadata_dir}/scene_listeners.train.json', 'exp_folder': './exp'}


In [15]:
# Load listener audiograms and songs
listener_dict = Listener.load_listener_dict(config.path.listeners_file)

with Path(config.path.gains_file).open("r", encoding="utf-8") as file:
    gains = json.load(file)

with Path(config.path.scenes_file).open("r", encoding="utf-8") as file:
    scenes = json.load(file)

with Path(config.path.scene_listeners_file).open("r", encoding="utf-8") as file:
    scenes_listeners = json.load(file)

with Path(config.path.music_file).open("r", encoding="utf-8") as file:
    songs = json.load(file)

Load the enhancer (NAL-R) and compression setting (OFF)

In [16]:
enhancer = NALR(**config.nalr)
compressor = Compressor(**config.compressor)

Create a list of songs, listeners (audiogram) and head position (hrtf) to generate, or evaluate the mix - based on provided data

In [17]:
# Select a batch to process
scene_listener_pairs = make_scene_listener_list(
    scenes_listeners, config.evaluate.small_test
)

scene_listener_pairs = scene_listener_pairs[
    config.evaluate.batch :: config.evaluate.batch_size
]

In [18]:
print(scene_listener_pairs[0])

('scene_10001', 'L0066')


# Process Audio - Currently set to process a fixed number of runs (listener - scene pairings, can be changed to run entire dataset)

In [19]:
previous_song = ""
num_scenes = len(scene_listener_pairs)
nrun = 1 # change to num_scenes for full dataset

for idx in range(nrun):
    scene_id, listener_id = scene_listener_pairs[idx]

    scene = scenes[scene_id]
    song_name = f"{scene['music']}-{scene['head_loudspeaker_positions']}"

    logger.info(
        f"[{idx:03d}/{num_scenes:03d}] "
        f"Processing {scene_id}: {song_name} for listener {listener_id}"
    )
    # Get the listener's audiogram
    listener = listener_dict[listener_id]


    # Read the binaural mixture, preprocess and then demix


    # Convert to 32-bit floating point and transpose
    # from [samples, channels] to [channels, samples]
    if song_name != previous_song:
        mixture_signal = read_signal(
            filename=Path(config.path.music_dir)
            / songs[song_name]["Path"]
            / "mixture.wav",
            sample_rate=config.sample_rate,
            allow_resample=True,
        )

        # demix
        stems: dict[str, ndarray] = decompose_signal(
            model=separation_model,
            model_sample_rate=model_sample_rate,
            signal=mixture_signal,
            signal_sample_rate=config.sample_rate,
            device=device,
            sources_list=sources_order,
            listener=listener,
            normalise=normalise,
        )

        # Reweight the stems based on provided gains and downmix to stereo
        stems = apply_gains(stems, config.sample_rate, gains[scene["gain"]])
        enhanced_signal = remix_stems(stems, mixture_signal, model_sample_rate)

    # Enhance the signal: Process through Hearing Aid processing - NAL-R gain application and optional compression
    enhanced_signal = process_remix_for_listener(
    signal=enhanced_signal,
    enhancer=enhancer,
    compressor=compressor,
    listener=listener,
    apply_compressor=config.apply_compressor,
    )


    # Write to file! - Make sure the enhanced_folder directory is correct
    filename = Path(enhanced_folder) / f"{scene_id}_{listener.id}_remix.flac"

    filename.parent.mkdir(parents=True, exist_ok=True)
    save_flac_signal(
        signal=enhanced_signal,
        filename=filename,
        signal_sample_rate=config.sample_rate,
        output_sample_rate=config.remix_sample_rate,
        do_clip_signal=True,
        do_soft_clip=config.soft_clip,
        )

logger.info("Done!")